# Quick Start: Context Engineering Dashboard

Get up and running in 5 cells. This notebook shows how to:

1. Build a context trace manually
2. Visualize it in your notebook
3. Capture a **live OpenAI call** automatically
4. Compare two context strategies with a Sankey diff

In [1]:
# Setup: works with pip install OR local development
import sys
from pathlib import Path

# Try importing the package - if it fails, add parent directory to path
try:
    import context_engineering_dashboard
    print(f"Using installed package: {context_engineering_dashboard.__file__}")
except ImportError:
    # Running from local clone - add parent directory to path
    repo_root = Path().resolve().parent
    if repo_root not in sys.path:
        sys.path.insert(0, str(repo_root))
    print(f"Using local development: {repo_root}")

# Installation options (run one if package not found):
# Option 1: Install from GitHub release
# !pip install git+https://github.com/cp71-dlai/context-engineering-dashboard.git@v0.1.0

# Option 2: Install locally for development (from repo root)
# !pip install -e ".[dev,all]"

Using local development: /workspace


## 1 | Build a trace by hand

A **ContextTrace** is the core data structure. It holds a list of
**ContextComponents** (system prompt, user message, RAG docs, etc.)
and the model's context-window limit.

In [2]:
from context_engineering_dashboard import (
    ComponentType,
    ContextComponent,
    ContextTrace,
    ContextWindow,
)

components = [
    ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "You are a helpful coding assistant.", token_count=500),
    ContextComponent("rag1", ComponentType.RAG_DOCUMENT,  "ChromaDB stores embeddings for semantic search.", token_count=4200, metadata={"chroma_score": 0.93}),
    ContextComponent("rag2", ComponentType.RAG_DOCUMENT,  "Collections group related documents together.", token_count=2800, metadata={"chroma_score": 0.85}),
    ContextComponent("hist", ComponentType.CHAT_HISTORY,  "User previously asked about installation.", token_count=1100),
    ContextComponent("user", ComponentType.USER_MESSAGE,  "How do I query a Chroma collection?", token_count=350),
]

trace = ContextTrace(
    context_limit=128_000,
    components=components,
    total_tokens=sum(c.token_count for c in components),
)

print(f"Tokens: {trace.total_tokens:,} / {trace.context_limit:,}  ({trace.utilization:.1f}% used)")

Tokens: 8,950 / 128,000  (7.0% used)


## 2 | Visualize the context window

`ContextWindow` renders an interactive HTML widget right inside the notebook.
Each colored block represents one component, sized proportionally to its token count.

In [3]:
# Horizontal layout (default) -- hover blocks for details
ContextWindow(trace=trace)

In [6]:
# Treemap layout -- better for seeing small components
ContextWindow(trace=trace, layout="treemap")

## 3 | Trace a live OpenAI call

Wrap any `openai` call in `trace_openai()`. The tracer captures messages,
token usage, latency, and the response -- then builds the trace for you.

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()  # reads OPENAI_API_KEY from .env

True

In [8]:
from openai import OpenAI
from context_engineering_dashboard import trace_openai

client = OpenAI()  # uses OPENAI_API_KEY from environment

with trace_openai() as tracer:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a concise technical writer."},
            {"role": "user",   "content": "Explain what context engineering is and why it matters for LLM applications. Keep it to 3 sentences."},
        ],
        temperature=0.7,
    )

print("Response:", response.choices[0].message.content)
print()

openai_trace = tracer.result
print(f"Prompt tokens: {openai_trace.llm_trace.usage['prompt_tokens']}")
print(f"Completion tokens: {openai_trace.llm_trace.usage['completion_tokens']}")
print(f"Latency: {openai_trace.llm_trace.latency_ms:.0f} ms")

Response: Context engineering involves strategically designing and structuring the input prompts and surrounding data to optimize the performance of large language models (LLMs). It is crucial for LLM applications because the model's outputs are highly sensitive to the context provided, affecting the relevance, accuracy, and coherence of responses. Effective context engineering ensures that LLMs deliver more reliable and tailored results, enhancing user experience and application efficiency.

Prompt tokens: 39
Completion tokens: 79
Latency: 2675 ms


In [ ]:
# Visualize the captured trace -- click components to view, click text to edit
ContextWindow(trace=openai_trace)

## 4 | Compare two strategies with a Sankey diff

Imagine you refactored a prompt: trimmed chat history and dropped a RAG doc.
`ContextDiff` shows token flow between the two versions.

In [ ]:
from context_engineering_dashboard import ContextDiff

# "Before" -- verbose prompt
before = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "...", token_count=3000),
        ContextComponent("hist", ComponentType.CHAT_HISTORY,  "...", token_count=18000),
        ContextComponent("rag1", ComponentType.RAG_DOCUMENT,  "...", token_count=12000),
        ContextComponent("rag2", ComponentType.RAG_DOCUMENT,  "...", token_count=5000),
        ContextComponent("user", ComponentType.USER_MESSAGE,  "...", token_count=400),
    ],
    total_tokens=38400,
)

# "After" -- compacted prompt (summarized history, dropped low-score doc)
after = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "...", token_count=3000),
        ContextComponent("hist", ComponentType.CHAT_HISTORY,  "...", token_count=6000),
        ContextComponent("rag1", ComponentType.RAG_DOCUMENT,  "...", token_count=12000),
        ContextComponent("user", ComponentType.USER_MESSAGE,  "...", token_count=400),
    ],
    total_tokens=21400,
)

diff = ContextDiff(before=before, after=after, before_label="Verbose", after_label="Compacted")
diff.sankey()

In [ ]:
diff.summary()

## 5 | Save & reload traces

Traces serialize to JSON for reproducibility and sharing.

In [ ]:
openai_trace.to_json("quick_start_trace.json")

reloaded = ContextTrace.from_json("quick_start_trace.json")
print(f"Reloaded: {len(reloaded.components)} components, {reloaded.total_tokens:,} tokens")

# Clean up
import os
os.remove("quick_start_trace.json")

---

**Next steps:** See `deep_dive.ipynb` for Chroma RAG tracing, available-pool
visualization, and multi-query comparisons.